In [3]:
import numpy as np
import pandas as pd
import Ipynb_importer
import evaluation

importing Jupyter notebook from evaluation.ipynb


In [4]:
train_data=np.array(pd.read_csv('Data/train_data.csv',usecols=['userId','itemId','gerne']))
test_pd=pd.read_csv('Data/test_data.csv',usecols=['itemId','gerne'])
train_ui_data=np.array(pd.read_csv('generate_data/train_UIMatrix.csv'))
user_attr_data=np.array(pd.read_csv('generate_data/user_attribute.csv'))
user_attr_normal_data=np.array(pd.read_csv('generate_data/user_attribute_normal.csv'))

test_pd.drop_duplicates( keep='first', inplace=True)
test_data=np.array(test_pd)
test_ui_data=np.array(pd.read_csv('generate_data/test_UIMatrix.csv'))

In [5]:

def gen_train_data(train_data):
    for train in train_data:
        train[2]=train[2][1:-1]
    for train in train_data:
        gerne_list=np.int32(train[2].split(','))
        tmp=np.linspace (0,34,18)
        for g in gerne_list:
            tmp[g]=tmp[g]+1
        train[2]=np.array(tmp,dtype=np.int32)
    pd.DataFrame(train_data).to_csv('generate_data/train_data.csv',header=None,index=False)
     
def gen_test_data(test_data):
    itemId=[test[0] for test in test_data]
    for test in test_data:
        test[1]=test[1][1:-1]
    attr=[]
    for test in test_data:
        gerne_list=np.int32(test[1].split(','))
        tmp=np.linspace (0,34,18)
        for g in gerne_list:
            tmp[g]=tmp[g]+1
        attr.append(tmp)
    pd.DataFrame(itemId).to_csv( 'generate_data/test_item.csv',header=None,index =False)
    pd.DataFrame(attr).to_csv( 'generate_data/test_attribute.csv',header=None,index =False)

In [6]:
'''gen_train_data(train_data)
gen_test_data(test_data)'''

'gen_train_data(train_data)\ngen_test_data(test_data)'

In [7]:
train=np.array(pd.read_csv('generate_data/train_data.csv',header =None))
neg_data=np.array(pd.read_csv('generate_data/neg_data.csv'))
test_item=np.array(pd.read_csv('generate_data/test_item.csv').astype(np.int32))
test_attribute=np.array(pd.read_csv('generate_data/test_attribute.csv').astype(np.int32))

In [8]:
def shuffle_train():
    np.random.shuffle(train)
    
def shuffle_neg():
    np.random.shuffle(neg_data)
    
def get_train_data(start,end):
    data=train[start:end]
    user=[x[0] for x in data]
    item=[x[1] for x in data]
    attr=[x[2][1:-1].split() for x in data]
    user_attr_normal=user_attr_normal_data[user]
    return user,item,attr,user_attr_normal

def get_test_data():
    return test_item,test_attribute

def get_neg_data(start,end):
    data=neg_data[start:end]
    user=[x[0] for x in data]
    item=[x[1] for x in data]
    attr=[x[2][1:-1].split() for x in data]
    user_attr_normal=user_attr_normal_data[user]
    return user,item,attr,user_attr_normal

def get_intersection_similar_user(G_user,k):
    user_attr_matrixT=np.transpose(user_attr_data)    
    A=np.matmul(G_user,user_attr_matrixT)   
    intersection_rank_matrix=np.argsort(-A)
    return intersection_rank_matrix[:,0:k]

In [10]:
def test(test_item_batch, test_G_user):
    
    k_value = 20
    test_BATCH_SIZE = np.size(test_item_batch)
    
    test_intersection_similar_user = get_intersection_similar_user(test_G_user, k_value)
   
    count = 0
    for test_i, test_userlist in zip(test_item_batch, test_intersection_similar_user):       
        for test_u in test_userlist:
            
            if test_ui_data[test_u, test_i] == 1:
                count = count + 1            
    p_at_20 = round(count/(test_BATCH_SIZE * k_value), 4)
           
    ans = 0.0
    RS = []
    for test_i, test_userlist in zip(test_item_batch, test_intersection_similar_user):  
        r=[]
        for user in test_userlist:
            r.append(test_ui_data[user][test_i])
        RS.append(r)
    M_at_20 = evaluation.mean_average_precision(RS)
  
    ans = 0.0
    for test_i, test_userlist in zip(test_item_batch, test_intersection_similar_user):  
        r=[]
        for user in test_userlist:
            r.append(test_ui_data[user][test_i])
        ans = ans + evaluation.get_nDCG(r, k_value, method=1)
    G_at_20 = ans/test_BATCH_SIZE
    k_value = 10 
    
    count = 0
    for test_i, test_userlist in zip(test_item_batch, test_intersection_similar_user):       
        for test_u in test_userlist[:k_value]:
            
            if test_ui_data[test_u, test_i] == 1:
                count = count + 1            
    p_at_10 = round(count/(test_BATCH_SIZE * k_value), 4)
         
    ans = 0.0
    RS = []
    for test_i, test_userlist in zip(test_item_batch, test_intersection_similar_user):  
        r=[]
        for user in test_userlist[:k_value]:
            r.append(test_ui_data[user][test_i])
        RS.append(r)
    M_at_10 = evaluation.mean_average_precision(RS)
    

    ans = 0.0
    for test_i, test_userlist in zip(test_item_batch, test_intersection_similar_user):  
        r=[]
        for user in test_userlist[:k_value]:
            r.append(test_ui_data[user][test_i])
        ans = ans + evaluation.get_nDCG(r, k_value, method=1)
    G_at_10 = ans/test_BATCH_SIZE
    return p_at_10,p_at_20,M_at_10,M_at_20,G_at_10,G_at_20